In [1]:
import pandas as pd
import json
import os

# Class for Restructuring

#### Note for Restructuring

There are 7 columns.

| Column         | Description                         |
|----------------|-------------------------------------|
| id             | Unique identifier                   |
| title          | Title of research                   |
| category       | tuple (code, subject name, abbrev)  |
| authors        | List of authors                     |
| year           | Year of publication                 |
| abstracts      | Abstract text                       |
| references     | List of references (id of research) |


In [25]:
class utils:
    def id_apply(x):
        return x["prism:url"].split("/")[-1]
    
    def title_apply(x):
        if "dc:title" not in x or x["dc:title"] == None:
            return None
        return x["dc:title"]
    
    def author_apply(x):
        name_list = []
        for author in x["author"]:
            name_to_append = ""
            if "ce:given-name" in author:
                name_to_append = f"{author['ce:given-name']} {author['ce:surname']}"
            else: 
                name_to_append = author["ce:indexed-name"]
            name_list.append(name_to_append)
        return name_list
    
    def abstracts_apply(x):
        if "dc:description" not in x or x["dc:description"] == None:
            return None
        return x["dc:description"]
    
    def reference_apply(x):
        if x["bibrecord"] == None or x["bibrecord"]["tail"] == None or x["bibrecord"]["tail"]["bibliography"] == None or x["bibrecord"]["tail"]["bibliography"]["reference"] == None:
            return None
        reference_list = []
        temp = x["bibrecord"]["tail"]["bibliography"]["reference"]
        if type(temp) != type([]):
            temp = [temp]
        for ref in temp:
            if "ref-info" not in ref or "refd-itemidlist" not in ref["ref-info"]:
                continue
            if type(ref["ref-info"]["refd-itemidlist"]["itemid"]) == type([]):
                for j in ref["ref-info"]["refd-itemidlist"]["itemid"]:
                    if "@idtype" in j and j["@idtype"] == "SGR":
                        reference_list.append(j["$"])
            else:
                if "@idtype" in ref["ref-info"]["refd-itemidlist"]["itemid"] and ref["ref-info"]["refd-itemidlist"]["itemid"]["@idtype"] == "SGR":
                    reference_list.append(ref["ref-info"]["refd-itemidlist"]["itemid"]["$"])
        if len(reference_list) == 0:
            return None
        return reference_list
    
    def subject_apply(x):
        subject_list = []
        for subject in x["subject-area"]:
            name = subject["$"]
            code = subject["@code"]
            abbrev = subject["@abbrev"]
            subject_list.append((name, code, abbrev))
        return subject_list

In [26]:
class PrepareData:
    def __init__(self) -> None:
        self.data = None
        self.data_dir = "./data"
    
    def load_data(self):
        dfs = []
        dir_list = os.listdir("./data")
        cnt = 0
        for i in dir_list:
            if i == ".DS_Store":
                continue

            # Handle too many data
            # if not (i == "2018" or i == "2019"):
            #     continue
            
            sub_dir = os.path.join("./data", i)
            files = os.listdir(sub_dir)
            for f in files:
                if f == ".DS_Store":
                    continue

                file_path = os.path.join(sub_dir, f)
                with open(file_path, "r") as file:
                    json_data = json.load(file)

                row = {key: value for key, value in json_data["abstracts-retrieval-response"].items()}
                row["year"] = int(i)
                dfs.append(pd.DataFrame([row]))
                cnt += 1

        df = pd.concat(dfs, ignore_index=True)
        print("Total data: ", cnt)
        self.data = df

    def save_data(self):
        self.data.to_csv("data.csv", index=False)

    def restructure(self):
        df = self.data.copy()
        
        df["id"] = df["coredata"].apply(utils.id_apply)
        df["title"] = df["coredata"].apply(utils.title_apply)
        df["authors"] = df["authors"].apply(utils.author_apply)
        df["abstracts"] = df["coredata"].apply(utils.abstracts_apply)
        df["references"] = df["item"].apply(utils.reference_apply)
        df["category"] = df["subject-areas"].apply(utils.subject_apply)
        
        # Drop unnecessary columns
        df.drop(columns=["item", "affiliation", "coredata", "idxterms", "language", "authkeywords"], inplace=True)
        
        # Explode subject-areas
        df = df.explode("category", ignore_index=True)
        
        self.data = df

    def null_count(self):
        for c in self.data.columns:
            print(f"Column {c} have {self.data[c].isnull().sum()} null values")

In [27]:
data = PrepareData()

In [32]:
data.load_data()

Total data:  20216


In [33]:
print(f"Before restructure: {data.data.shape}")

Before restructure: (20216, 9)


In [34]:
data.data

,item,affiliation,coredata,idxterms,language,authkeywords,subject-areas,authors,year
0,{'ait:process-info': {'ait:status': {'@state':...,"[{'affiliation-city': 'Stanford', '@id': '6003...","{'srctype': 'b', 'eid': '2-s2.0-85077976956', ...",None,{'@xml:lang': 'eng'},None,"{'subject-area': [{'@_fa': 'true', '$': 'Medic...","{'author': [{'ce:given-name': 'Krit', 'preferr...",2018
1,{'ait:process-info': {'ait:status': {'@state':...,"{'affiliation-city': 'Bangkok', '@id': '600281...","{'srctype': 'p', 'eid': '2-s2.0-85060936020', ...","{'mainterm': [{'$': 'Antenna dimensions', '@we...",{'@xml:lang': 'eng'},None,"{'subject-area': [{'@_fa': 'true', '$': 'Elect...","{'author': [{'ce:given-name': 'Teerapong', 'pr...",2018
2,{'ait:process-info': {'ait:status': {'@state':...,"{'affiliation-city': 'Bangkok', '@id': '600281...","{'srctype': 'j', 'eid': '2-s2.0-85052201238', ...",{'mainterm': [{'$': 'Circulating fluidized bed...,{'@xml:lang': 'eng'},"{'author-keyword': [{'@_fa': 'true', '$': 'Cir...","{'subject-area': [{'@_fa': 'true', '$': 'Chemi...","{'author': [{'ce:given-name': 'Kiattikhoon', '...",2018
3,{'ait:process-info': {'ait:status': {'@state':...,"[{'affiliation-city': 'Hirosaki', '@id': '6003...","{'srctype': 'j', 'eid': '2-s2.0-85051498032', ...","{'mainterm': [{'$': 'Core-shell morphologies',...",{'@xml:lang': 'eng'},"{'author-keyword': [{'@_fa': 'true', '$': 'Enc...","{'subject-area': [{'@_fa': 'true', '$': 'Chemi...","{'author': [{'ce:given-name': 'Jittraporn', 'p...",2018
4,{'ait:process-info': {'ait:status': {'@state':...,"[{'affiliation-city': 'Bangkok', '@id': '60028...","{'srctype': 'j', 'eid': '2-s2.0-85050678366', ...","{'mainterm': [{'$': 'acpcPNA', '@weight': 'b',...",{'@xml:lang': 'eng'},"{'author-keyword': [{'@_fa': 'true', '$': 'acp...","{'subject-area': [{'@_fa': 'true', '$': 'Analy...","{'author': [{'ce:given-name': 'Prinjaporn', 'p...",2018
...,...,...,...,...,...,...,...,...,...
20211,{'ait:process-info': {'ait:status': {'@state':...,"[{'affiliation-city': 'Bangkok', '@id': '60028...","{'srctype': 'j', 'eid': '2-s2.0-85111945558', ...","{'mainterm': [{'$': 'High selectivity', '@weig...",{'@xml:lang': 'eng'},"{'author-keyword': [{'@_fa': 'true', '$': 'Lon...","{'subject-area': [{'@_fa': 'true', '$': 'Catal...","{'author': [{'ce:given-name': 'Duy', 'preferre...",2023
20212,{'ait:process-info': {'ait:status': {'@state':...,"{'affiliation-city': 'Bangkok', '@id': '600281...","{'srctype': 'j', 'eid': '2-s2.0-85111408415', ...","{'mainterm': [{'$': 'Analytical tool', '@weigh...",{'@xml:lang': 'eng'},"{'author-keyword': [{'@_fa': 'true', '$': 'Bio...","{'subject-area': [{'@_fa': 'true', '$': 'Analy...","{'author': [{'ce:given-name': 'Waleed', 'prefe...",2023
20213,{'ait:process-info': {'ait:status': {'@state':...,"[{'affiliation-city': 'Bangkok', '@id': '60028...","{'srctype': 'j', 'eid': '2-s2.0-85110903700', ...",None,{'@xml:lang': 'eng'},"{'author-keyword': [{'@_fa': 'true', '$': 'con...","{'subject-area': [{'@_fa': 'true', '$': 'Educa...","{'author': [{'ce:given-name': 'Tejendra', 'pre...",2023
20214,{'ait:process-info': {'ait:status': {'@state':...,"[{'affiliation-city': 'Bangkok', '@id': '60028...","{'srctype': 'j', 'eid': '2-s2.0-85106740832', ...",None,{'@xml:lang': 'eng'},"{'author-keyword': [{'@_fa': 'true', '$': 'Ana...","{'subject-area': [{'@_fa': 'true', '$': 'Ecolo...","{'author': [{'ce:given-name': 'Ratchaneegorn',...",2023


In [35]:
data.restructure()

In [36]:
print(f"After restructure: {data.data.shape}")

After restructure: (50064, 8)


In [37]:
data.data

,subject-areas,authors,year,id,title,abstracts,references,category
0,"{'subject-area': [{'@_fa': 'true', '$': 'Medic...","[Krit Pongpirul, Matthew P. Lungren]",2018,85077976956,Public health and international epidemiology f...,None,"[0002667983, 33750367977, 85013970385, 7795302...","(Medicine (all), 2700, MEDI)"
1,"{'subject-area': [{'@_fa': 'true', '$': 'Elect...","[Teerapong Pratumsiri, Panuwat Janpugdee]",2018,85060936020,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,"[85006043726, 85046336244, 85060914424, 850463...","(Electrical and Electronic Engineering, 2208, ..."
2,"{'subject-area': [{'@_fa': 'true', '$': 'Elect...","[Teerapong Pratumsiri, Panuwat Janpugdee]",2018,85060936020,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,"[85006043726, 85046336244, 85060914424, 850463...","(Electronic, Optical and Magnetic Materials, 2..."
3,"{'subject-area': [{'@_fa': 'true', '$': 'Chemi...","[Kiattikhoon Phuakpunk, Benjapon Chalermsinsuw...",2018,85052201238,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,"[2942655685, 84908055658, 85052199786, 8485971...","(Chemistry (all), 1600, CHEM)"
4,"{'subject-area': [{'@_fa': 'true', '$': 'Chemi...","[Kiattikhoon Phuakpunk, Benjapon Chalermsinsuw...",2018,85052201238,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,"[2942655685, 84908055658, 85052199786, 8485971...","(Chemical Engineering (all), 1500, CENG)"
...,...,...,...,...,...,...,...,...
50059,"{'subject-area': [{'@_fa': 'true', '$': 'Ecolo...","[Ratchaneegorn Mapanao, Wirat Jiwyam, Nudtha N...",2023,85106740832,Effects of black soldier fly (Hermetia illucen...,The effects of replacing fish meal protein wit...,"[84942036010, 0009764501, 84949970831, 8505962...","(Ecology, 2303, ENVI)"
50060,"{'subject-area': [{'@_fa': 'true', '$': 'Ecolo...","[Ratchaneegorn Mapanao, Wirat Jiwyam, Nudtha N...",2023,85106740832,Effects of black soldier fly (Hermetia illucen...,The effects of replacing fish meal protein wit...,"[84942036010, 0009764501, 84949970831, 8505962...","(Aquatic Science, 1104, AGRI)"
50061,"{'subject-area': [{'@_fa': 'true', '$': 'Geogr...",[Vatana Chea],2023,85106046890,Effects of remittances on household poverty an...,The author uses data from the Cambodia Socio-E...,"[36749086437, 54349107987, 0024885310, 3374585...","(Geography, Planning and Development, 3305, SOCI)"
50062,"{'subject-area': [{'@_fa': 'true', '$': 'Geogr...",[Vatana Chea],2023,85106046890,Effects of remittances on household poverty an...,The author uses data from the Cambodia Socio-E...,"[36749086437, 54349107987, 0024885310, 3374585...","(Development, 3303, SOCI)"


In [38]:
data.data.sample(5)

,subject-areas,authors,year,id,title,abstracts,references,category
3017,"{'subject-area': [{'@_fa': 'true', '$': 'Micro...","[Nathita Phumthanakorn, Punpichaya Fungwithaya...",2018,85048036785,Enterotoxin gene profile of methicillin-resist...,Purpose. This study aimed to detect and identi...,"[84864147868, 79955962281, 84925483592, 338455...","(Microbiology (medical), 2726, MEDI)"
4200,"{'subject-area': [{'@_fa': 'true', '$': 'Anato...","[Eun Jin Woo, Hyunwoo Jung, Tanvaa Tansatit]",2018,85045222768,Cranial index in a modern people of Thai ancestry,The present research aims to examine the crani...,"[84895168539, 84941189307, 84914151295, 002532...","(Histology, 2722, MEDI)"
17142,"{'subject-area': [{'@_fa': 'true', '$': 'Ceram...","[Tidarut Jirawattanasomkul, Suched Likitlersua...",2020,85082879133,Structural behaviour of pre-damaged reinforced...,This paper examines the use of natural jute fa...,"[85046683437, 85050191643, 84974574251, 849035...","(Ceramics and Composites, 2503, MATE)"
46202,"{'subject-area': [{'@_fa': 'true', '$': 'Finan...","[Nopparat Wongsinhirun, Pattanaporn Chatjutham...",2023,85148492722,Corporate culture and board gender diversity: ...,Exploiting a distinctive measure of corporate ...,"[85148937971, 70349151758, 84880633653, 848569...","(Economics and Econometrics, 2002, ECON)"
12333,"{'subject-area': [{'@_fa': 'true', '$': 'Mater...","[Thanaporn Wilairat, Nattawut Saechin, Wantane...",2019,85079839423,Effects of hot molding parameters on physical ...,Hot molding is one of the most important proce...,"[33748773315, 85032568173, 84868330109, 799537...","(Mechanical Engineering, 2210, ENGI)"


In [39]:
data.null_count()

Column subject-areas have 0 null values
Column authors have 0 null values
Column year have 0 null values
Column id have 0 null values
Column title have 2 null values
Column abstracts have 1251 null values
Column references have 899 null values
Column category have 0 null values


In [40]:
data.data.dropna(subset=["abstracts"], inplace=True)
data.data.dropna(subset=["references"], inplace=True)
data.data.dropna(subset=["title"], inplace=True)

data.data.shape

(48227, 8)

In [41]:
data.data

,subject-areas,authors,year,id,title,abstracts,references,category
1,"{'subject-area': [{'@_fa': 'true', '$': 'Elect...","[Teerapong Pratumsiri, Panuwat Janpugdee]",2018,85060936020,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,"[85006043726, 85046336244, 85060914424, 850463...","(Electrical and Electronic Engineering, 2208, ..."
2,"{'subject-area': [{'@_fa': 'true', '$': 'Elect...","[Teerapong Pratumsiri, Panuwat Janpugdee]",2018,85060936020,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,"[85006043726, 85046336244, 85060914424, 850463...","(Electronic, Optical and Magnetic Materials, 2..."
3,"{'subject-area': [{'@_fa': 'true', '$': 'Chemi...","[Kiattikhoon Phuakpunk, Benjapon Chalermsinsuw...",2018,85052201238,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,"[2942655685, 84908055658, 85052199786, 8485971...","(Chemistry (all), 1600, CHEM)"
4,"{'subject-area': [{'@_fa': 'true', '$': 'Chemi...","[Kiattikhoon Phuakpunk, Benjapon Chalermsinsuw...",2018,85052201238,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,"[2942655685, 84908055658, 85052199786, 8485971...","(Chemical Engineering (all), 1500, CENG)"
5,"{'subject-area': [{'@_fa': 'true', '$': 'Chemi...","[Kiattikhoon Phuakpunk, Benjapon Chalermsinsuw...",2018,85052201238,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,"[2942655685, 84908055658, 85052199786, 8485971...","(Industrial and Manufacturing Engineering, 220..."
...,...,...,...,...,...,...,...,...
50059,"{'subject-area': [{'@_fa': 'true', '$': 'Ecolo...","[Ratchaneegorn Mapanao, Wirat Jiwyam, Nudtha N...",2023,85106740832,Effects of black soldier fly (Hermetia illucen...,The effects of replacing fish meal protein wit...,"[84942036010, 0009764501, 84949970831, 8505962...","(Ecology, 2303, ENVI)"
50060,"{'subject-area': [{'@_fa': 'true', '$': 'Ecolo...","[Ratchaneegorn Mapanao, Wirat Jiwyam, Nudtha N...",2023,85106740832,Effects of black soldier fly (Hermetia illucen...,The effects of replacing fish meal protein wit...,"[84942036010, 0009764501, 84949970831, 8505962...","(Aquatic Science, 1104, AGRI)"
50061,"{'subject-area': [{'@_fa': 'true', '$': 'Geogr...",[Vatana Chea],2023,85106046890,Effects of remittances on household poverty an...,The author uses data from the Cambodia Socio-E...,"[36749086437, 54349107987, 0024885310, 3374585...","(Geography, Planning and Development, 3305, SOCI)"
50062,"{'subject-area': [{'@_fa': 'true', '$': 'Geogr...",[Vatana Chea],2023,85106046890,Effects of remittances on household poverty an...,The author uses data from the Cambodia Socio-E...,"[36749086437, 54349107987, 0024885310, 3374585...","(Development, 3303, SOCI)"


In [42]:
data.save_data()